In [ ]:
import json
import requests

# --- Config for MultiChain JSON-RPC ---
RPC_USER = 'multichainrpc'
RPC_PASSWORD = '785fZEXDLjKD1EB4xjXger9HkRK78MDa6eTRMquKFwpt'
CHAIN_NAME = 'secret'
RPC_PORT = 6792  # Replace with your MultiChain port

URL = f'http://127.0.0.1:{RPC_PORT}'
HEADERS = {'content-type': 'application/json'}

def multichain_rpc(method, params=[]):
    payload = {
        "method": method,
        "params": params,
        "id": 1,
        "chain_name": CHAIN_NAME
    }
    response = requests.post(URL, data=json.dumps(payload), headers=HEADERS, auth=(RPC_USER, RPC_PASSWORD))

    if response.status_code == 200:
        result = response.json()
        if 'error' in result and result['error'] is not None:
            print(f"❌ RPC Error ({method}):", result['error'])
            return None
        return result['result']
    else:
        print(f"❌ HTTP Error ({method}):", response.status_code, response.text)
        return None


# Caesar Cipher Functions
def caesar_encrypt(text, shift):
    result = ''
    for char in text:
        if char.isalpha():
            offset = 65 if char.isupper() else 97
            result += chr((ord(char) - offset + shift) % 26 + offset)
        else:
            result += char
    return result

def caesar_decrypt(text, shift):
    return caesar_encrypt(text, -shift)

# Load address_links.json
with open("network.json", "r") as f:
    data = json.load(f)

master = data['master_address']
node_A = data['linked_addresses'][0]['address']
node_B = data['linked_addresses'][1]['address']

# Message and Caesar blinding
message = "Patient X has diabetes"
shift = 5
blinded = caesar_encrypt(message, shift)
signed_blinded = f"{blinded} [signed_by:{master}]"

# Create stream if not exists
try:
    multichain_rpc("create", ["stream", "blind_transfers", True])
    multichain_rpc("subscribe", ["blind_transfers"])
except:
    pass  # Stream may already exist

# Publish to stream
txid = multichain_rpc("publish", [
    "blind_transfers",
    node_B,
    {
        "json": {
            "from": node_A,
            "to": node_B,
            "blinded_signed": signed_blinded,
            "cipher": "Caesar",
            "shift": shift
        }
    }
])

# Print success
print(f"\n✅ Message sent from {node_A} to {node_B} (blinded) and published by signer {master}")
print(f"🔏 Caesar Blinded Msg: {signed_blinded}")
print(f"📨 TXID: {txid}")

#Sir kindly see the multichain output below in another cell


✅ Message sent from 13xnXAJysKKqk97mSWoRXBF9NJ3u8ZDH8Havgr to 1K1N7zH1ErysKyuFstj6fd63JDKy1CgsCgqPJP (blinded) and published by signer 19TVh7M9UhbaVtHFKRszrmMoCfLBJmAc2RcUwa
🔏 Caesar Blinded Msg: Ufynjsy C mfx infgjyjx [signed_by:19TVh7M9UhbaVtHFKRszrmMoCfLBJmAc2RcUwa]
📨 TXID: a4a658bc4ed28b4b862f54ed06a21e7953258a0f6bf052433943b3d68b421452


In [ ]:
# ` multichain-cli secret liststreamkeyitems blind_transfers "1K1N7zH1ErysKyuFstj6fd63JDKy1CgsCgqPJP"
# {"method":"liststreamkeyitems","params":["blind_transfers","1K1N7zH1ErysKyuFstj6fd63JDKy1CgsCgqPJP"],"id":"23203222-1745064065","chain_name":"secret"}

# [
#     {
#         "publishers" : [
#             "13xnXAJysKKqk97mSWoRXBF9NJ3u8ZDH8Havgr"
#         ],
#         "keys" : [
#             "1K1N7zH1ErysKyuFstj6fd63JDKy1CgsCgqPJP"
#         ],
#         "offchain" : false,
#         "available" : true,
#         "data" : {
#             "json" : {
#                 "from" : "13xnXAJysKKqk97mSWoRXBF9NJ3u8ZDH8Havgr",
#                 "to" : "1K1N7zH1ErysKyuFstj6fd63JDKy1CgsCgqPJP",
#                 "blinded_signed" : "Ufynjsy C mfx infgjyjx [signed_by:19TVh7M9UhbaVtHFKRszrmMoCfLBJmAc2RcUwa]",
#                 "cipher" : "Caesar",

# [
#     {
#         "publishers" : [
#             "13xnXAJysKKqk97mSWoRXBF9NJ3u8ZDH8Havgr"
#         ],
#         "keys" : [
#             "1K1N7zH1ErysKyuFstj6fd63JDKy1CgsCgqPJP"
#         ],
#         "offchain" : false,
#         "available" : true,
#         "data" : {
#             "json" : {
#                 "from" : "13xnXAJysKKqk97mSWoRXBF9NJ3u8ZDH8Havgr",
#                 "to" : "1K1N7zH1ErysKyuFstj6fd63JDKy1CgsCgqPJP",
#                 "blinded_signed" : "Ufynjsy C mfx infgjyjx [signed_by:19TVh7M9UhbaVtHFKRszrmMoCfLBJmAc2RcUwa]",
#                 "cipher" : "Caesar",
#             "1K1N7zH1ErysKyuFstj6fd63JDKy1CgsCgqPJP"
#         ],
#         "offchain" : false,
#         "available" : true,
#         "data" : {
#             "json" : {
#                 "from" : "13xnXAJysKKqk97mSWoRXBF9NJ3u8ZDH8Havgr",
#                 "to" : "1K1N7zH1ErysKyuFstj6fd63JDKy1CgsCgqPJP",
#                 "blinded_signed" : "Ufynjsy C mfx infgjyjx [signed_by:19TVh7M9UhbaVtHFKRszrmMoCfLBJmAc2RcUwa]",
#                 "cipher" : "Caesar",
#         "available" : true,
#         "data" : {
#             "json" : {
#                 "from" : "13xnXAJysKKqk97mSWoRXBF9NJ3u8ZDH8Havgr",
#                 "to" : "1K1N7zH1ErysKyuFstj6fd63JDKy1CgsCgqPJP",
#                 "blinded_signed" : "Ufynjsy C mfx infgjyjx [signed_by:19TVh7M9UhbaVtHFKRszrmMoCfLBJmAc2RcUwa]",
#                 "cipher" : "Caesar",
#             "json" : {
#                 "from" : "13xnXAJysKKqk97mSWoRXBF9NJ3u8ZDH8Havgr",
#                 "to" : "1K1N7zH1ErysKyuFstj6fd63JDKy1CgsCgqPJP",
#                 "blinded_signed" : "Ufynjsy C mfx infgjyjx [signed_by:19TVh7M9UhbaVtHFKRszrmMoCfLBJmAc2RcUwa]",
#                 "cipher" : "Caesar",
#                 "to" : "1K1N7zH1ErysKyuFstj6fd63JDKy1CgsCgqPJP",
#                 "blinded_signed" : "Ufynjsy C mfx infgjyjx [signed_by:19TVh7M9UhbaVtHFKRszrmMoCfLBJmAc2RcUwa]",
#                 "cipher" : "Caesar",
#                 "cipher" : "Caesar",
#                 "shift" : 5
#             }
#         },
#         "confirmations" : 0,
#         "txid" : "a4a658bc4ed28b4b862f54ed06a21e7953258a0f6bf052433943b3d68b421452"
#     }
# ]
# PS C:\Users\Monis\Desktop\BlockChain_and_its_applications>







#                 "shift" : 5
#             }
#         },
#         "confirmations" : 0,
#         "txid" : "a4a658bc4ed28b4b862f54ed06a21e7953258a0f6bf052433943b3d68b421452"
#     }
# ]
# PS C:\Users\Monis\Desktop\BlockChain_and_its_applications>




#                 "shift" : 5
#             }
#         },
#         "confirmations" : 0,
#         "txid" : "a4a658bc4ed28b4b862f54ed06a21e7953258a0f6bf052433943b3d68b421452"
#     }
# ]
# PS C:\Users\Monis\Desktop\BlockChain_and_its_applications>


#                 "shift" : 5
#             }
#         },
#         "confirmations" : 0,
#         "txid" : "a4a658bc4ed28b4b862f54ed06a21e7953258a0f6bf052433943b3d68b421452"
#     }
# ]
# PS C:\Users\Monis\Desktop\BlockChain_and_its_applications>
#                 "shift" : 5
#             }
#         },
#         "confirmations" : 0,
#         "txid" : "a4a658bc4ed28b4b862f54ed06a21e7953258a0f6bf052433943b3d68b421452"
#     }
#                 "shift" : 5
#             }
#         },
#                 "shift" : 5
#                 "shift" : 5
#             }
#         },
#         "confirmations" : 0,
#         "txid" : "a4a658bc4ed28b4b862f54ed06a21e7953258a0f6bf052433943b3d68b421452"
#     }
# ]